In [1]:
import pandas as pd
!pip install tweepy
import tweepy
import numpy
import warnings
#In order to ignore warnings when overwriting row values in panda dataframes
warnings.filterwarnings('ignore')

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
from google.colab import drive 
drive.mount('/content/drive')

Mounted at /content/drive


####Tweepy credentials

In [ ]:
CONSUMER_KEY = #
CONSUMER_SECRET = #
OAUTH_TOKEN = #
OAUTH_TOKEN_SECRET = #

In [ ]:
auth = tweepy.OAuthHandler(CONSUMER_KEY, CONSUMER_SECRET)
auth.set_access_token(OAUTH_TOKEN, OAUTH_TOKEN_SECRET)
api = tweepy.API(auth, wait_on_rate_limit=True)

#DISCARDED DATA

In [ ]:
#Large Scale Crowdsourcing and Characterization of Twitter Abusive Behavior
df_abusive_text = pd.read_csv('/content/hatespeech_id_label_founta.csv', encoding='utf8', header= None)

###[Large Scale Crowdsourcing and Characterization of Twitter Abusive Behavior](https://arxiv.org/pdf/1802.00393.pdf)

####Obtaining tweet text

In [ ]:
#Split the single column into 2
df_abusive_text[['TweetID', 'hate speech']] = df_abusive_text[0].str.split(',', 1, expand=True)
df_abusive_text = df_abusive_text[['TweetID', 'hate speech']]

In [ ]:
df_abusive_text['text'] = "0"

In [ ]:
for idx, row in df_abusive_text.iterrows():
  #This condition is for when execution is stopped and later on resumed
  if not any(c.isalpha() for c in str(df_abusive_text['text'][idx])):
    try:
      if api.get_status(df_abusive_text['TweetID'][idx]):
        tweetFetched = api.get_status(df_abusive_text['TweetID'][idx])
        df_abusive_text['text'][idx] = tweetFetched.text
    except tweepy.TweepError as e:
      # print (e.api_code)
      # print (getExceptionMessage(e.reason))
      df_abusive_text['text'][idx] = "TWITTER__ERROR" + getExceptionMessage(e.reason)
    #We save the dataframe every line to not lose any progress (in case this cell would stop after 6+ hours!)
    df_abusive_text.to_csv('/content/drive/MyDrive/TFG/data/abusive_text_data.csv', index=False)  

####Rest of preprocessing


In [3]:
df_abusive_text = pd.read_csv('/content/drive/MyDrive/TFG/data/abusive_text_data.csv', encoding='utf8', engine="python")

In [4]:
df_abusive_text

,TweetID,hate speech,text
0,849667487180259329,abusive,TWITTER__ERRORNo status found with that ID.
1,850490912954351616,abusive,Alex Brosas another idiot #ALDUBKSGoesToUS ht...
2,848791766853668864,abusive,"RT @ItIzBiz: as Nancy Reagan would say, 'just ..."
3,848306464892604416,abusive,TWITTER__ERRORNo status found with that ID.
4,850010509969465344,normal,RT @MailOnline: The Nazi death gas so horrific...
...,...,...,...
100002,848922176128376832,hateful,0
100003,849405078956109829,abusive,0
100004,847981251143254016,normal,0
100005,850180983253073920,abusive,0


In [5]:
for idx, row in df_abusive_text.iterrows():
  if "TWITTER__ERROR" in str(row.text) or "0" in str(row.text) or row["hate speech"]!="hateful":
    df_abusive_text = df_abusive_text.drop(labels=idx, axis=0)

In [6]:
df_abusive_text

,TweetID,hate speech,text
5,850433664890544128,hateful,I hate er chase because if the Bitch that work...
12,850100742035836929,hateful,RT @nyctophil3: Pineapples do not belong on pi...
21,847804507367100416,hateful,Niggas keep talking about women wearing weave ...
33,850090319165050880,hateful,@vappywave idiot that's not gonna work. you go...
67,848686686930382848,hateful,RT @ayevonnn: bruh i fucking hate people like ...
...,...,...,...
44055,849005550524387328,hateful,lrt is cute because they're opposite elements ...
44088,850157792925241346,hateful,Voted for the biggest fucking liar @POTUS ever...
44348,848578067043889153,hateful,"Supposed ""Anti-fascist"" favors murderous commu..."
44371,850378195224465411,hateful,Syrian ally Iran blasts U.S. missile strikes a...
